<a href="https://colab.research.google.com/github/casjunior93/Projeto-DIO---Analise-de-Sentimentos-com-Language-Studio-no-Azure-AI/blob/main/An%C3%A1lise_de_sentimentos_no_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando o SDK para análise de texto

Execute na primeira vez que iniciar o notebook. Depois pode comentar colocando um # na frente

In [ ]:
!pip install azure-ai-textanalytics==5.2.0

Agora importamos o que vamos precisar usar do SDK:

In [3]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

Informe aqui a chave de assinatura e o ponto final (endpoint). Podem ser obtidos na página do recurso em "Chaves e Ponto Ponto Final" no Portal do Azure, ou, dentro do Estúdio de linguagem", em Language Studio > Classify text > Analyze sentiment and mine opinions, no final da página.

In [41]:
language_key = "SUA CHAVE"
language_endpoint = "SEU PONTO FINAL"

Agora vamos nos autenticar usando a chave e o ponto final:

In [42]:
def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint,
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

Essa é a função para detectar sentimentos e opiniões que vamos utilizar:

In [32]:
def sentiment_analysis_with_opinion_mining_example(client, documents):

    ''' documents = [
        "The food and service were unacceptable. The concierge was nice, however."
    ] '''

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
    negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]

    positive_mined_opinions = []
    mixed_mined_opinions = []
    negative_mined_opinions = []

    for document in doc_result:
        print("Sentimento do documento: {}".format(document.sentiment))
        print("Pontuações gerais: Positivo={0:.2f}; Neutro={1:.2f}; Negativo={2:.2f} \n".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))
        for sentence in document.sentences:
            print("Sentença: {}".format(sentence.text))
            print("Sentimento da sentença: {}".format(sentence.sentiment))
            print("Pontuação da sentença:\nPositivo={0:.2f}\nNeutro={1:.2f}\nNegativo={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                print("......'{}' target '{}'".format(target.sentiment, target.text))
                print("......Pontuação alvo:\n......Positivo={0:.2f}\n......Negativo={1:.2f}\n".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))
                for assessment in mined_opinion.assessments:
                    print("......'{}' assessment '{}'".format(assessment.sentiment, assessment.text))
                    print("......Pontuação da avaliação:\n......Positivo={0:.2f}\n......Negativo={1:.2f}\n".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))
        print("\n")
        print('------------------------------------')

Vamos criar uma lista com opiniões:

In [9]:
texto_positivo = "Estou encantado com meu novo Lenovo G40-70! Sua velocidade impressionante e design elegante superaram minhas expectativas. A tela HD é nítida, proporcionando uma experiência visual incrível. O teclado é confortável, e a bateria tem uma durabilidade surpreendente. Além disso, o processador oferece desempenho excepcional para multitarefas. Estou muito satisfeito com minha compra e definitivamente recomendo o Lenovo G40-70 a quem procura um laptop confiável e eficiente."

In [10]:
texto_negativo = "Infelizmente, minha experiência com o Lenovo G40-70 tem sido decepcionante. A performance do processador não atende às minhas necessidades, frequentemente apresentando lentidão em tarefas simples. A duração da bateria também é abaixo da média, exigindo recargas frequentes. A construção do laptop parece frágil, e alguns componentes parecem de qualidade inferior. Além disso, o aquecimento excessivo é uma preocupação constante. Infelizmente, não posso recomendar o Lenovo G40-70 com base na minha experiência insatisfatória."

In [11]:
texto_neutro = "O Lenovo G40-70 oferece um desempenho razoável para tarefas diárias. A tela e o teclado são aceitáveis, mas não impressionam. A durabilidade da bateria é mediana, exigindo recargas frequentes. O design é simples, e a construção parece sólida o suficiente. O processador atende às necessidades básicas, mas pode apresentar lentidão em multitarefas. A relação custo-benefício é equilibrada, considerando o preço. Em resumo, o Lenovo G40-70 é uma opção aceitável para usuários casuais, mas pode deixar a desejar para aqueles que buscam desempenho e recursos mais avançados."

In [26]:
documents = [
    texto_positivo,
    texto_negativo,
    texto_neutro
]

Agora chamamos o método de análise passando os textos:

In [40]:
sentiment_analysis_with_opinion_mining_example(client, documents)

ClientAuthenticationError: (401) The Request text analysis over a collection of documents. Operation under Microsoft Cognitive Language Service is not supported with the current subscription key and pricing tier SpeechServices.S0.
Code: 401
Message: The Request text analysis over a collection of documents. Operation under Microsoft Cognitive Language Service is not supported with the current subscription key and pricing tier SpeechServices.S0.

O erro acima ocorreu pois esse serviço não é suportado para minha chave de inscrição e tipo de preço atual.

Para resolver, criei outro recurso de linguagem utilizando o Escalão de preço
Free F0 (5K Transactions per 30 days). Lembre-se de executar a função de autenticação novamente após atualizar a chave e o ponto final.

Testando novamente:

In [24]:
sentiment_analysis_with_opinion_mining_example(client, documents)

Document Sentiment: mixed
Overall scores: positive=0.43; neutral=0.04; negative=0.53 

Sentence: The food and service were unacceptable. 
Sentence sentiment: negative
Sentence score:
Positive=0.00
Neutral=0.01
Negative=0.99

......'negative' target 'food'
......Target score:
......Positive=0.01
......Negative=0.99

......'negative' assessment 'unacceptable'
......Assessment score:
......Positive=0.01
......Negative=0.99

......'negative' target 'service'
......Target score:
......Positive=0.01
......Negative=0.99

......'negative' assessment 'unacceptable'
......Assessment score:
......Positive=0.01
......Negative=0.99



Sentence: The concierge was nice, however.
Sentence sentiment: positive
Sentence score:
Positive=0.86
Neutral=0.08
Negative=0.07

......'positive' target 'concierge'
......Target score:
......Positive=1.00
......Negative=0.00

......'positive' assessment 'nice'
......Assessment score:
......Positive=1.00
......Negative=0.00







Sucesso!